# Mod 6 Homework Reflections 
# Weeks 5-8

In [3]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 43.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import seaborn as sns
import networkx as nx
from sklearn.metrics import accuracy_score, roc_auc_score


# Week 5 

1. Draw a diagram for the following negative feedback loop:

Sweating causes body temperature to decrease.  High body temperature causes sweating.

A negative feedback loop means that one thing increases another while the second thing decreases the first.

Remember that we are using directed acyclic graphs where two things cannot directly cause each other.

2. Describe an example of a positive feedback loop.  This means that one things increases another while the second things also increases the first.

3. Draw a diagram for the following situation:

Lightning storms frighten away deer and bears, decreasing their population, and cause flowers to grow, increasing their population.
Bears eat deer, decreasing their population.
Deer eat flowers, decreasing their population.

Write a dataset that simulates this situation.  (Show the code.) Include noise / randomness in all cases.

Identify a backdoor path with one or more confounders for the relationship between deer and flowers.

4. Draw a diagram for a situation of your own invention.  The diagram should include at least four nodes, one confounder, and one collider.  Be sure that it is acyclic (no loops).  Which node would say is most like a treatment (X)?  Which is most like an outcome (Y)?


In [5]:
# Question 3

# Simulating the data 

np.random.seed(42)  
n = 500  # observations

# Exogenous variable: Lightning storms, frequency/intensity 
lightning = np.random.normal(5, 2, n)

# Bears are frightened away by lightning
bears = 100 - 8 * lightning + np.random.normal(0, 5, n)

# Deer are frightened by lightning and eaten by bears
deer = 80 - 5 * lightning - 0.3 * bears + np.random.normal(0, 5, n)

# Flowers: grow more with lightning, eaten by deer
flowers = 50 + 4 * lightning - 0.5 * deer + np.random.normal(0, 5, n)

df = pd.DataFrame({
    "lightning": lightning,
    "bears": bears,
    "deer": deer,
    "flowers": flowers
})

print(df.head())

   lightning      bears       deer    flowers
0   5.993428  56.683461  40.024597  57.853220
1   4.723471  71.759312  39.478018  46.398948
2   6.295377  42.644146  36.028023  53.076502
3   8.046060  38.446368  25.001107  69.666813
4   4.531693  60.493241  42.684678  45.933511


# Week 6

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

df6 = pd.read_csv("homework_6.1.csv")

# Create treated and untreated groups
treated = df6[df6["X"] == 1].reset_index(drop=True)
untreated = df6[df6["X"] == 0].reset_index(drop=True)

# Define nearest-neighbor match function
def nearest_match(source, target):
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(target[["Z"]])
    distances, indices = nn.kneighbors(source[["Z"]])
    return indices.flatten()

# Match treated to untreated and untreated to treated
treated_to_untreated_idx = nearest_match(treated, untreated)
untreated_to_treated_idx = nearest_match(untreated, treated)

# Generate counterfactual outcomes
treated["Y_cf"] = untreated.loc[treated_to_untreated_idx, "Y"].values
untreated["Y_cf"] = treated.loc[untreated_to_treated_idx, "Y"].values

# Compute treatment effects
treated["TE"] = treated["Y"] - treated["Y_cf"]
untreated["TE"] = treated.loc[untreated_to_treated_idx, "Y"].values - untreated["Y"]

# Compute average treatment effects
ate = (treated["TE"].mean() + untreated["TE"].mean()) / 2
att = treated["TE"].mean()
atu = untreated["TE"].mean()

# Compute Optimal Treatment Effect (max) and MTE using the 90th percentile
optimal_te = untreated["TE"].max()
MTE_90th = np.percentile(untreated["TE"], 90)

# Print results
print(f"Average Treatment Effect (ATE): {ate:.4f}")
print(f"Average Treatment Effect on Treated (ATT): {att:.4f}")
print(f"Average Treatment Effect on Untreated (ATU): {atu:.4f}")
print(f"Naive Optimal Treatment Effect (max): {optimal_te:.4f}")
print(f"Robust Marginal Treatment Effect (90th percentile): {MTE_90th:.4f}")


Average Treatment Effect (ATE): 1.6979
Average Treatment Effect on Treated (ATT): 1.8464
Average Treatment Effect on Untreated (ATU): 1.5495
Naive Optimal Treatment Effect (max): 2.1725
Robust Marginal Treatment Effect (90th percentile): 1.9280


# Week 7

In [2]:
# Week 7, Question 1

import numpy as np
import statsmodels.api as sm

np.random.seed(0)
n = 1000

# Confounder
W = np.random.normal(0, 1, n)

# Treatment
X = W + np.random.normal(0, 1, n)

# Outcome
Y = 2*X + 3*W + np.random.normal(0, 1, n)

# Regress Y and X - no confounder)
model_no_conf = sm.OLS(Y, sm.add_constant(X)).fit()

# Regress Y and X - including confounder
model_full = sm.OLS(Y, sm.add_constant(np.column_stack([X, W]))).fit()

print("True effect of X on Y: 2.0")
print("Without Confounder W:", model_no_conf.params[1])
print("With Confounder W:", model_full.params[1])


True effect of X on Y: 2.0
Without Confounder W: 3.510405682088037
With Confounder W: 1.9981919455639527


In [7]:
# Week 7, Question 2

np.random.seed(0)

p_values = []  

for i in range(1000):
    n = 200  # number of data points

    W = np.random.normal(0, 1, n)
    X = np.random.normal(0, 1, n)
    error = np.random.normal(0, 1, n)

    # true model, coeff of W is 0
    Y = 2 * X + error  

    # regression: including X and W
    XW = sm.add_constant(np.column_stack((X, W)))
    model = sm.OLS(Y, XW).fit()

    # p-value of W is the 3rd value (after constant and X)
    p_w = model.pvalues[2]
    p_values.append(p_w)

# results
min_p = np.min(p_values)
count_sig = np.sum(np.array(p_values) < 0.05)

print("Smallest p-value for W:", round(min_p, 6))
print("Number of times p < 0.05:", count_sig, "out of 1000")

Smallest p-value for W: 0.002831
Number of times p < 0.05: 51 out of 1000


# Week 8

In [4]:
df8 = pd.read_csv("homework_8.1.csv")
df8.head()

,Unnamed: 0,X,Y,Z
0,0,1,4.109218,1.764052
1,1,0,2.259504,0.400157
2,2,0,-0.647584,0.978738
3,3,0,2.106071,2.240893
4,4,1,3.583464,1.867558


In [ ]:
# Week 8, Question 1

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

df8 = pd.read_csv("homework_8.1.csv")

Z = df8[["Z"]]
X = df8["X"]
Y = df8["Y"]

log_model = LogisticRegression()
log_model.fit(Z, X)

P = log_model.predict_proba(Z)[:, 1]

# IPW
weights = np.where(X == 1, 1 / P, 1 / (1 - P))

ATE = (np.sum(weights * (X * Y)) / np.sum(weights * X)) - \
      (np.sum(weights * ((1 - X) * Y)) / np.sum(weights * (1 - X)))

print(f"ATE ≈ {ATE:.3f}")


ATE ≈ 2.274


In [7]:
# Question 2

print(np.round(P[:3], 2))

[0.84 0.58 0.71]


In [6]:
# Question 3


from scipy.spatial.distance import mahalanobis
from numpy.linalg import inv

df82 = pd.read_csv("homework_8.2.csv")

# treated (X=1), untreated (X=0)
treated = df82[df82["X"] == 1].reset_index(drop=True)
untreated = df82[df82["X"] == 0].reset_index(drop=True)

Z = df82[["Z1", "Z2"]].to_numpy()

cov_matrix = np.cov(Z, rowvar=False)
inv_cov = inv(cov_matrix)

# matching using Mahalanobis distance
matches = []
for i in range(len(treated)):
    z_treated = treated.loc[i, ["Z1", "Z2"]].to_numpy()
    distances = [mahalanobis(z_treated, untreated.loc[j, ["Z1", "Z2"]].to_numpy(), inv_cov)
                 for j in range(len(untreated))]
    nearest_index = np.argmin(distances)
    matches.append(untreated.loc[nearest_index, "Y"])

treated["Y_cf"] = matches
treated["TE"] = treated["Y"] - treated["Y_cf"]

ATE = treated["TE"].mean()
print(f"ATE ≈ {ATE:.2f}")


ATE ≈ 3.44


In [8]:
# Question 4

import pandas as pd
import numpy as np
from numpy.linalg import inv, pinv
from scipy.spatial.distance import cdist

df = pd.read_csv("homework_8.2.csv")

treated = df[df["X"] == 1].reset_index(drop=True)
untreated = df[df["X"] == 0].reset_index(drop=True)

Z_all = df[["Z1", "Z2"]].to_numpy()
Z_t = treated[["Z1", "Z2"]].to_numpy()
Z_u = untreated[["Z1", "Z2"]].to_numpy()

cov = np.cov(Z_all, rowvar=False)
VI = pinv(cov)

D = cdist(Z_t, Z_u, metric="mahalanobis", VI=VI)

nearest_dists = D.min(axis=1)
nearest_idxs  = D.argmin(axis=1)

i_farthest = int(nearest_dists.argmax())

z1, z2 = treated.loc[i_farthest, ["Z1", "Z2"]].to_numpy()

print("Treated item with least common support (exact):", (z1, z2))
print("Rounded to match options:", (np.round(z1, 1), np.round(z2, 1)))

j_nearest = int(nearest_idxs[i_farthest])
z1_u, z2_u = untreated.loc[j_nearest, ["Z1", "Z2"]].to_numpy()
print("Nearest untreated to that treated (Z1, Z2):", (z1_u, z2_u))
print("Mahalanobis distance to nearest untreated:", float(nearest_dists[i_farthest]))

Treated item with least common support (exact): (np.float64(2.69622405256358), np.float64(0.5381554886023228))
Rounded to match options: (np.float64(2.7), np.float64(0.5))
Nearest untreated to that treated (Z1, Z2): (np.float64(1.5199948607657727), np.float64(-1.2822079376259403))
Mahalanobis distance to nearest untreated: 1.3830045328325058
